In [1]:
import pandas as pd
import os
import glob
import numpy as np
import tensorflow as tf
import matplotlib
from tensorflow import keras
from keras.models import load_model
import sklearn
from sklearn.datasets import make_blobs
import sys

2023-08-31 10:20:13.717504: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 10:20:13.719507: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 10:20:13.772238: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 10:20:13.773985: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 10:20:14.424419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### Model Import

In [7]:
model = load_model('/workspace/Ishan/P1_confDeepILST/data_decomp/oldscript/NNdecomp.h5')

### Import COM Mapped AA Force Profiles

In [ ]:
path_a = "/workspace/Ishan/P1_confDeepILST/data_decomp/AAForce/CH4/AAforceWF.txt"

path_b = "/workspace/Ishan/P1_confDeepILST/data_decomp/AAForce/CH4/AAforceFF.txt"

path_c = "/workspace/Ishan/P1_confDeepILST/data_decomp/AAForce/CH4/Training_Norm.txt"

def antinormalize(norarray):                                                                                           
 simarray = norarray * (norm_array[2,1] - norm_array[2,0]) + norm_array[2,0]
 return simarray

csv_files = sorted(glob.glob(path_a))
force_list = []
for f in csv_files:
  g = pd.read_csv(f, delimiter = ' ', skiprows=None, header=None)
  force_list.append(g)
  WFforce_array = np.array(force_list)     

csv_files = sorted(glob.glob(path_b))
force_list = []
for f in csv_files:
  g = pd.read_csv(f, delimiter = ' ', skiprows=None, header=None)
  force_list.append(g)
  FFforce_array = np.array(force_list)  # array containing force data

csv_files = sorted(glob.glob(path_c))
norm_list = []
for f in csv_files:
  g = pd.read_csv(f, delimiter = ' ', header=None, usecols=[0,1], index_col=None)
  norm_list.append(g)
  norm_array = np.array(norm_list)

norm_array = np.squeeze(norm_array) 

### Inference on Trained Model to get CG parameters

In [17]:
forceWF = WFforce_array[:,:,0]
forceFF = FFforce_array[:,:,0]

normforceWF = (forceWF - norm_array[0,0] )/(norm_array[0,1] - norm_array[0,0])
normforceFF = (forceFF - norm_array[1,0] )/(norm_array[1,1] - norm_array[1,0])

X_test= np.concatenate((normforceWF, normforceFF), axis=1)
result_test = model.predict(X_test)

x = result_test[:,0] * norm_array[3,0]                               
y = result_test[:,1]
a = result_test[:,2] * norm_array[3,1]                               
b = result_test[:,3]  

x = x.reshape(len(x),1)
y = y.reshape(len(y),1)
a = a.reshape(len(a),1)
b = b.reshape(len(b),1)

z = np.concatenate((x, y, a, b), axis=1) 
LJ_multiatom = antinormalize(z) 

print("FF Epsilon:", LJ_multiatom[0,0])
print("FF Sigma:", LJ_multiatom[0,1])
print("WF Epsilon:", LJ_multiatom[0,2])
print("WF Sigma:", LJ_multiatom[0,3])

1/1 [==============================] - 0s 25ms/step
FF Epsilon: 0.33118653
FF Sigma: 3.6058395
WF Epsilon: 0.15241715
WF Sigma: 3.5902967
